<a href="https://colab.research.google.com/github/mtjon/infompr-group-project/blob/all_in_one/all_in_one.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For this notebook elemets were taken from:
1. https://www.tensorflow.org/text/tutorials/nmt_with_attention, and;
2. https://www.tensorflow.org/text/tutorials/transformer

For the RNN + GRU, bi-directional layer was removed. For the transformer, all sub-classing has been copied, as well as custom learning schedule.

Data-preprocessing has been adopted to fit the dataset used, which can be found here:
- Data loading: https://github.com/EagleW/Writing-editing-Network/blob/master/split_data.py
- Dataset: https://github.com/EagleW/ACL_titles_abstracts_dataset

We've manually adjusted the dataset to remove the empty rows between title-abstract pairs, and adjusted the data loading code to work with the changes accordingly.

# Installing and loading Required packages

In [ ]:
# Install the most re version of TensorFlow to use the improved
# masking support for `tf.keras.layers.MultiHeadAttention`.
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2
!pip uninstall -y -q tensorflow keras tensorflow-estimator tensorflow-text
!pip install -q tensorflow_datasets
# !pip install -q -U tensorflow-text==2.9.* tensorflow==2.9.*
!pip install -q -U tensorflow-text==2.9.* tensorflow==2.9.*
!pip install einops

In [ ]:
!pip install -q -U tensorflow-text tensorflow

In [ ]:
!pip list | grep tensorflow

In [ ]:
import logging
import time

import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds
import tensorflow as tf

import tensorflow_text
import tensorflow_text as tf_text

# For data loading and shuffling
import random
from random import shuffle

# for shape checking -> used in RNN and GRU, see class below
import einops

In [ ]:
#@title
class ShapeChecker():
  def __init__(self):
    # Keep a cache of every axis-name seen
    self.shapes = {}

  def __call__(self, tensor, names, broadcast=False):
    if not tf.executing_eagerly():
      return

    parsed = einops.parse_shape(tensor, names)

    for name, new_dim in parsed.items():
      old_dim = self.shapes.get(name, None)
      
      if (broadcast and new_dim == 1):
        continue

      if old_dim is None:
        # If the axis name is new, add its length to the cache.
        self.shapes[name] = new_dim
        continue

      if new_dim != old_dim:
        raise ValueError(f"Shape mismatch for dimension: '{name}'\n"
                         f"    found: {new_dim}\n"
                         f"    expected: {old_dim}\n")

# Data Loading

## Functions

In [ ]:
# #Necessary when upload fails (Firefox+Colab issue)
# from google.colab import files
# uploaded = files.upload()

In [ ]:
# modified data externally to remove all blank lines
def write_data(f, ext, mod):
    # Code van https://github.com/EagleW/Writing-editing-Network/blob/master/split_data.py
    file1=open(f, 'r')
    lines=file1.readlines()
    print(len(lines))
    file1.close()
    abs_t = []
    abstracts = []
    titles = []
    i = 0
    # TODO: possibly generates wrong tibs/abs when encounter
    for line in lines:
        if i % mod == 0:
            titles.append(line)
        elif i % mod == 1:
            abstracts.append(line)
        i += 1
    for i in range(len(abstracts)):
        if len(titles[i]) > 0 and len(abstracts[i]) > 0:
            h_a_pair = (titles[i], abstracts[i])
            abs_t.append(h_a_pair)
    shuffle(abs_t)
    total = len(abs_t)
    dev = total//10
    train  = total - dev - dev
    i = 0
    file1=open("val{}.txt".format(ext), 'w')
    for i in range(dev):
        file1.writelines(abs_t[i][0])
        file1.writelines(abs_t[i][1])
    #    file1.writelines("\n")
    file1.close()
    file1=open("test{}.txt".format(ext), 'w')
    for i in range(dev, 2 * dev):
        file1.writelines(abs_t[i][0])
        file1.writelines(abs_t[i][1])
    #    file1.writelines("\n")
    file1.close()
    file1=open("train{}.txt".format(ext), 'w')
    for i in range(2 * dev, total):
        file1.writelines(abs_t[i][0])
        file1.writelines(abs_t[i][1])
    #    file1.writelines("\n")
    file1.close()

def get_abs_and_titles_from_raw(path_to_file):
    abstracts, titles = [], []
    abscount, titcount = 0, 0
    with open(path_to_file) as data:
        lines = data.readlines()
        print(len(lines))
        for abs in lines[1::2]:
            abstracts.append(abs.strip())
            abscount += 1
        for title in lines[0::2]:
            # // TODO: check if we need to add start and end tokens
            titles.append(title.strip())
            titcount+=1
            # titles.append('[START] ' + title.strip() + ' [END]')
    print(abscount, titcount)
    # print(titles[-1],'\n',titles[-2])
    return abstracts, titles

## Loading data

Make sure to have the modified dataset uploaden (i.e., ACL title+abs data set with empty rows removed).

### Writing and loading text files

In [ ]:
write_data("acl_titles_and_abstracts_mod.txt", "", 2) # writes train, val, test.txt

In [ ]:
train_text, train_labels = get_abs_and_titles_from_raw('train.txt')
val_text, val_labels = get_abs_and_titles_from_raw('val.txt')
test_text, test_labels = get_abs_and_titles_from_raw('test.txt')

### Sanity checks

In [ ]:
print(train_labels[0])
print(train_text[0])

In [ ]:
print(val_labels[0])
print(val_text[0])

In [ ]:
print(test_labels[0])
print(test_text[0])

### Batching

In [ ]:
BATCH_SIZE = 64
train_dataset = tf.data.Dataset.from_tensor_slices((train_text, train_labels)).batch(BATCH_SIZE)
val_dataset = tf.data.Dataset.from_tensor_slices((val_text, val_labels)).batch(BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices((test_text, test_labels)).batch(BATCH_SIZE)

In [ ]:
counter = 0
for abs, tit in train_dataset.take(1):
    print('> Abstract examples:')
    for i, a in enumerate(abs.numpy()):
        print(i+1, a.decode('utf-8'))
        
    print()
    
    print('> Title examples:')
    for i, t in enumerate(tit.numpy()):
        print(i+1, t.decode('utf-8'))
    
    counter += 1
    if counter == 10:
        break

# Preprocessing

In [ ]:
def title_preprocessor(text):
    text = tf.strings.lower(text)
    text = tf.strings.strip(text)
    text = tf.strings.join(['[START]', text, '[END]'], separator=' ')
    return text

def abstract_preprocessor(text):
    text = tf.strings.lower(text)
    text = tf.strings.strip(text)
    # Added start and end tokens. read somewhere its important for the RNNs
    # not for transformer.. Idk.
    # - Damion
    text = tf.strings.join(['[START]', text, '[END]'], separator=' ')
    return text

# def abstract_preprocessor_mk2(text):
#     # from: https://www.tensorflow.org/text/tutorials/nmt_with_attention#text_preprocessing
#     # I've included this below code for preprocessing, since in the abstract,
#     # punctuation etc isnt important - perhaps. Only for the titles. 
#     # So, perhaps performance increase
#     # - Damion

#     # Additionally, the start and end token seem to do something
#     # for the RNN's?
#     # Split accente d characters.
    
#     #text = tf_text.normalize_utf8(text, 'NFKD')
#     #text = tf.strings.lower(text)
    
#     # Keep space, a to z, and select punctuation.
#     #text = tf.strings.regex_replace(text, '[^ a-z.?!,¿]', '')
#     text = tf.strings.regex_replace(text, '[^ a-z]', '')
    
#     # Add spaces around punctuation.
#     #text = tf.strings.regex_replace(text, '[.?!,¿]', r' \0 ')
    
#     # Strip whitespace.
#     text = tf.strings.strip(text)

#     text = tf.strings.join(['[START]', text, '[END]'], separator=' ')
#     return text

abs_text_processor = tf.keras.layers.TextVectorization(
    standardize=abstract_preprocessor,
    ragged=True,
    output_mode='int'
    )

# abs_text_processor = tf.keras.layers.TextVectorization(
#     standardize=abstract_preprocessor_mk2,
#     ragged=True,
#     output_mode='int'
#     )

tit_text_processor = tf.keras.layers.TextVectorization(
    standardize=title_preprocessor,
    ragged=True,
    output_mode='int'
    )

abs_text_processor.adapt(train_dataset.map(lambda abs, tit: abs))
tit_text_processor.adapt(train_dataset.map(lambda abs, tit: tit))

def process_text(context, target):
    context = abs_text_processor(context).to_tensor()
    target = tit_text_processor(target)
    targ_in = target[:,:-1].to_tensor()
    targ_out = target[:,1:].to_tensor()
    return (context, targ_in), targ_out


train_ds = train_dataset.map(process_text, tf.data.AUTOTUNE)
val_ds = val_dataset.map(process_text, tf.data.AUTOTUNE)
test_ds = test_dataset.map(process_text, tf.data.AUTOTUNE)



In [ ]:
for (ex_context_tok, ex_tar_in), ex_tar_out in train_ds.take(1):
    print(ex_context_tok[0, :10].numpy()) 
    print()
    print(ex_tar_in[0, :10].numpy()) 
    print(ex_tar_out[0, :10].numpy())

# Models + components

## RNN + GRU components

In [ ]:
class EncoderRNN(tf.keras.layers.Layer):
  def __init__(self, text_processor, units, GRU=False):
    super(EncoderRNN, self).__init__()
    self.text_processor = text_processor
    self.vocab_size = text_processor.vocabulary_size()
    self.units = units

    # The embedding layer converts tokens to vectors
    self.embedding = tf.keras.layers.Embedding(self.vocab_size, units,
                                               mask_zero=True)

    # The RNN layer processes those vectors sequentially.
    # self.rnn = tf.keras.layers.Bidirectional(
    #     merge_mode='sum',
    #     layer=tf.keras.layers.GRU(units,
    #                         # Return the sequence and state
    #                         return_sequences=True,
    #                         recurrent_initializer='glorot_uniform'))
    if not GRU:
        self.rnn = (tf.keras.layers \
                    .SimpleRNN(units,
                    # Return the sequence and state
                    return_sequences=True,
                    recurrent_initializer='glorot_uniform'))
    else:
        self.rnn = (tf.keras.layers
                    .GRU(units,
                    # Return the sequence and state
                    return_sequences=True,
                    recurrent_initializer='glorot_uniform'))

  def call(self, x):
    shape_checker = ShapeChecker()
    shape_checker(x, 'batch s')

    # 2. The embedding layer looks up the embedding vector for each token.
    x = self.embedding(x)
    shape_checker(x, 'batch s units')

    # 3. The GRU processes the sequence of embeddings.
    x = self.rnn(x)
    shape_checker(x, 'batch s units')

    # 4. Returns the new sequence of embeddings.
    return x

  def convert_input(self, texts):
    texts = tf.convert_to_tensor(texts)
    if len(texts.shape) == 0:
      texts = tf.convert_to_tensor(texts)[tf.newaxis]
    context = self.text_processor(texts).to_tensor()
    context = self(context)
    return context

In [ ]:
class CrossAttentionRNN(tf.keras.layers.Layer):
  def __init__(self, units, **kwargs):
    super().__init__()
    self.mha = tf.keras.layers.MultiHeadAttention(key_dim=units, num_heads=1, **kwargs)
    self.layernorm = tf.keras.layers.LayerNormalization()
    self.add = tf.keras.layers.Add()

  def call(self, x, context):
    shape_checker = ShapeChecker()

    shape_checker(x, 'batch t units')
    shape_checker(context, 'batch s units')

    attn_output, attn_scores = self.mha(
        query=x,
        value=context,
        return_attention_scores=True)

    shape_checker(x, 'batch t units')
    shape_checker(attn_scores, 'batch heads t s')

    # Cache the attention scores for plotting later.
    attn_scores = tf.reduce_mean(attn_scores, axis=1)
    shape_checker(attn_scores, 'batch t s')
    self.last_attention_weights = attn_scores

    x = self.add([x, attn_output])
    x = self.layernorm(x)

    return x

In [ ]:
class DecoderRNN(tf.keras.layers.Layer):
    @classmethod
    def add_method(cls, fun):
        setattr(cls, fun.__name__, fun)
        return fun

    def __init__(self, text_processor, units, GRU=True):
        super(DecoderRNN, self).__init__()
        self.text_processor = text_processor
        self.vocab_size = text_processor.vocabulary_size()
        self.word_to_id = tf.keras.layers.StringLookup(
            vocabulary=text_processor.get_vocabulary(),
            mask_token='', oov_token='[UNK]')
        self.id_to_word = tf.keras.layers.StringLookup(
            vocabulary=text_processor.get_vocabulary(),
            mask_token='', oov_token='[UNK]',
            invert=True)
        self.start_token = self.word_to_id('[START]')
        self.end_token = self.word_to_id('[END]')
        print('Did start- and end-token get stored properly?')
        print(self.start_token, self.end_token)

        self.units = units


        # 1. The embedding layer converts token IDs to vectors
        self.embedding = tf.keras.layers.Embedding(self.vocab_size,
                                                    units, mask_zero=True)

        # 2. The RNN keeps track of what's been generated so far.
        if not GRU:
            self.rnn = (tf.keras.layers \
                        .SimpleRNN(units,
                        # Return the sequence and state
                        return_sequences=True,
                        return_state=True,
                        recurrent_initializer='glorot_uniform'))
        else:
            self.rnn = (tf.keras.layers \
                        .GRU(units,
                        # Return the sequence and state
                        return_sequences=True,
                        return_state=True,
                        recurrent_initializer='glorot_uniform'))

        # 3. The RNN output will be the query for the attention layer.
        self.attention = CrossAttentionRNN(units)

        # 4. This fully connected layer produces the logits for each
        # output token.
        self.output_layer = tf.keras.layers.Dense(self.vocab_size)

    def call(self,
            context, x,
            state=None,
            return_state=False):  
        shape_checker = ShapeChecker()
        shape_checker(x, 'batch t')
        shape_checker(context, 'batch s units')

        # 1. Lookup the embeddings
        x = self.embedding(x)
        shape_checker(x, 'batch t units')

        # 2. Process the target sequence.
        x, state = self.rnn(x, initial_state=state)
        shape_checker(x, 'batch t units')

        # 3. Use the RNN output as the query for the attention over the context.
        x = self.attention(x, context)
        self.last_attention_weights = self.attention.last_attention_weights
        shape_checker(x, 'batch t units')
        shape_checker(self.last_attention_weights, 'batch t s')

        # Step 4. Generate logit predictions for the next token.
        logits = self.output_layer(x)
        shape_checker(logits, 'batch t target_vocab_size')

        if return_state:
            return logits, state
        else:
            return logits

    def get_initial_state(self, context):
        batch_size = tf.shape(context)[0]
        start_tokens = tf.fill([batch_size, 1], self.start_token)
        done = tf.zeros([batch_size, 1], dtype=tf.bool)
        embedded = self.embedding(start_tokens)
        return start_tokens, done, self.rnn.get_initial_state(embedded)[0]
    
    def tokens_to_text(self, tokens):
        words = self.id_to_word(tokens)
        result = tf.strings.reduce_join(words, axis=-1, separator=' ')
        result = tf.strings.regex_replace(result, '^ *\[START\] *', '')
        result = tf.strings.regex_replace(result, ' *\[END\] *$', '')
        return result

    def get_next_token(self, context, next_token, done, state, temperature = 0.0):
        logits, state = self(
        context, next_token,
        state = state,
        return_state=True) 

        if temperature == 0.0:
            next_token = tf.argmax(logits, axis=-1)
        else:
            logits = logits[:, -1, :]/temperature
            next_token = tf.random.categorical(logits, num_samples=1)

        # If a sequence produces an `end_token`, set it `done`
        done = done | (next_token == self.end_token)
        # Once a sequence is done it only produces 0-padding.
        next_token = tf.where(done, tf.constant(0, dtype=tf.int64), next_token)

        return next_token, done, state

In [ ]:
class Translator(tf.keras.Model):
    @classmethod
    def add_method(cls, fun):
        setattr(cls, fun.__name__, fun)
        return fun

    def __init__(self, units,
                context_text_processor,
                target_text_processor,
                GRU=True):
        super().__init__()
        # Build the encoder and decoder
        # GRU False will result in SimpleRNN instead of GRU
        self.encoder = EncoderRNN(context_text_processor, units, GRU=GRU)
        self.decoder = DecoderRNN(target_text_processor, units, GRU=GRU)

    def call(self, inputs):
        context, x = inputs
        context = self.encoder(context)
        logits = self.decoder(context, x)

        #TODO(b/250038731): remove this
        try:
            # Delete the keras mask, so keras doesn't scale the loss+accuracy. 
            del logits._keras_mask
        except AttributeError:
            pass

        return logits
    
    def translate(self,
                texts, *,
                max_length=50,
                temperature=0.0):
        # Process the input texts
        context = self.encoder.convert_input(texts)
        batch_size = tf.shape(texts)[0]

        # Setup the loop inputs
        tokens = []
        attention_weights = []
        next_token, done, state = self.decoder.get_initial_state(context)

        for _ in range(max_length):
            # Generate the next token
            next_token, done, state = self.decoder.get_next_token(
                context, next_token, done,  state, temperature)

            # Collect the generated tokens
            tokens.append(next_token)
            attention_weights.append(self.decoder.last_attention_weights)

            if tf.executing_eagerly() and tf.reduce_all(done):
                break

        # Stack the lists of tokens and attention weights.
        tokens = tf.concat(tokens, axis=-1)   # t*[(batch 1)] -> (batch, t)
        self.last_attention_weights = tf.concat(attention_weights, axis=1)  # t*[(batch 1 s)] -> (batch, t s)

        result = self.decoder.tokens_to_text(tokens)
        return result

## Transformer Component

In [ ]:
class PositionalEmbeddingT(tf.keras.layers.Layer):
    def __init__(self, vocab_size, d_model):
        super().__init__()
        self.d_model = d_model
        self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=True)
        self.pos_encoding = self.positional_encoding(length=2048, depth=d_model)
    
    def compute_mask(self, *args, **kwargs):
        return self.embedding.compute_mask(*args, **kwargs)
    
    def call(self, x):
        length = tf.shape(x)[1]
        x = self.embedding(x)
        # This factor sets the relative scale of the embedding and positonal_encoding.
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x = x + self.pos_encoding[tf.newaxis, :length, :]
        return x
    
    def positional_encoding(self, length, depth):
        depth = depth/2

        positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
        depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)

        angle_rates = 1 / (10000**depths)         # (1, depth)
        angle_rads = positions * angle_rates      # (pos, depth)

        pos_encoding = np.concatenate(
            [np.sin(angle_rads), np.cos(angle_rads)],
            axis=-1) 

        return tf.cast(pos_encoding, dtype=tf.float32)

class BaseAttentionT(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__()
        self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
        self.layernorm = tf.keras.layers.LayerNormalization()
        self.add = tf.keras.layers.Add()

class CrossAttentionT(BaseAttentionT):
    def call(self, x, context):
        attn_output, attn_scores = self.mha(
            query=x,
            key=context,
            value=context,
            return_attention_scores=True)
        # Cache the attention scores for plotting later.
        self.last_attn_scores = attn_scores
        x = self.add([x, attn_output])
        x = self.layernorm(x)
        return x

class GlobalSelfAttentionT(BaseAttentionT):
    def call(self, x):
        attn_output = self.mha(
            query=x,
            value=x,
            key=x)
        x = self.add([x, attn_output])
        x = self.layernorm(x)
        return x

class CausalSelfAttentionT(BaseAttentionT):
    def call(self, x):
        attn_output = self.mha(
            query=x,
            value=x,
            key=x,
            use_causal_mask = True)
        x = self.add([x, attn_output])
        x = self.layernorm(x)
        return x

class FeedForwardT(tf.keras.layers.Layer):
    def __init__(self, d_model, dff, dropout_rate=0.1):
        super().__init__()
        self.seq = tf.keras.Sequential([
            tf.keras.layers.Dense(dff, activation='relu'),
            tf.keras.layers.Dense(d_model),
            tf.keras.layers.Dropout(dropout_rate)
        ])
        self.add = tf.keras.layers.Add()
        self.layer_norm = tf.keras.layers.LayerNormalization()
    def call(self, x):
        x = self.add([x, self.seq(x)])
        x = self.layer_norm(x)
        return x

In [ ]:
class EncoderLayerT(tf.keras.layers.Layer):
    def __init__(self,*, d_model, num_heads, dff, dropout_rate=0.1):
        super().__init__()

        self.self_attention = GlobalSelfAttentionT(
            num_heads=num_heads,
            key_dim=d_model,
            dropout=dropout_rate)

        self.ffn = FeedForwardT(d_model, dff)

    def call(self, x):
        x = self.self_attention(x)
        x = self.ffn(x)
        return x

class EncoderT(tf.keras.layers.Layer):
    def __init__(self, *, num_layers, d_model, num_heads,
                dff, vocab_size, dropout_rate=0.1):
        super().__init__()

        # Wanneer we translator maken moeten we het volgende gebruiken:
        # self.text_processor = text_processor
        # self.vocab_size = vocab_size

        self.d_model = d_model
        self.num_layers = num_layers

        self.pos_embedding = PositionalEmbeddingT(
            vocab_size=vocab_size, d_model=d_model)

        self.enc_layers = [
            EncoderLayerT(d_model=d_model,
                            num_heads=num_heads,
                            dff=dff,
                            dropout_rate=dropout_rate)
            for _ in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(dropout_rate)

    def call(self, x):
        # `x` is token-IDs shape: (batch, seq_len)
        x = self.pos_embedding(x)  # Shape `(batch_size, seq_len, d_model)`.

        # Add dropout.
        x = self.dropout(x)

        for i in range(self.num_layers):
            x = self.enc_layers[i](x)

        return x  # Shape `(batch_size, seq_len, d_model)`.

    def convert_input(self, texts):
        pass

In [ ]:
class DecoderLayerT(tf.keras.layers.Layer):
  def __init__(self,
               *,
               d_model,
               num_heads,
               dff,
               dropout_rate=0.1):
    super(DecoderLayerT, self).__init__()

    self.causal_self_attention = CausalSelfAttentionT(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)
    
    self.cross_attention = CrossAttentionT(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.ffn = FeedForwardT(d_model, dff)

  def call(self, x, context):
    x = self.causal_self_attention(x=x)
    x = self.cross_attention(x=x, context=context)

    # Cache the last attention scores for plotting later
    self.last_attn_scores = self.cross_attention.last_attn_scores

    x = self.ffn(x)  # Shape `(batch_size, seq_len, d_model)`.
    return x

class DecoderT(tf.keras.layers.Layer):
    def __init__(self, *, num_layers, d_model, num_heads, dff, vocab_size,
                dropout_rate=0.1):
        super(DecoderT, self).__init__()
        ### TODO:
        # word_to_id etc, zie RNN decoder van Nick en Jan

        self.d_model = d_model
        self.num_layers = num_layers

        self.pos_embedding = PositionalEmbeddingT(vocab_size=vocab_size,
                                                    d_model=d_model)
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        self.dec_layers = [
            DecoderLayerT(d_model=d_model, num_heads=num_heads,
                            dff=dff, dropout_rate=dropout_rate)
            for _ in range(num_layers)]

        self.last_attn_scores = None

    def call(self, x, context):
        # `x` is token-IDs shape (batch, target_seq_len)
        x = self.pos_embedding(x)  # (batch_size, target_seq_len, d_model)

        x = self.dropout(x)

        for i in range(self.num_layers):
            x  = self.dec_layers[i](x, context)

        self.last_attn_scores = self.dec_layers[-1].last_attn_scores

        # The shape of x is (batch_size, target_seq_len, d_model).
        return x

In [ ]:
class Transformer(tf.keras.Model):
    def __init__(self, *, num_layers, d_model, num_heads, dff,
                input_vocab_size, target_vocab_size, dropout_rate=0.1):
        super().__init__()
        self.encoder = EncoderT(num_layers=num_layers, d_model=d_model,
                                num_heads=num_heads, dff=dff,
                                vocab_size=input_vocab_size,
                                dropout_rate=dropout_rate)

        self.decoder = DecoderT(num_layers=num_layers, d_model=d_model,
                                num_heads=num_heads, dff=dff,
                                vocab_size=target_vocab_size,
                                dropout_rate=dropout_rate)

        self.final_layer = tf.keras.layers.Dense(target_vocab_size)
    
    def call(self, inputs):
        # To use a Keras model with `.fit` you must pass all your inputs in the
        # first argument.
        context, x = inputs

        context = self.encoder(context)  # (batch_size, context_len, d_model)

        x = self.decoder(x, context)  # (batch_size, target_len, d_model)

        # Final linear layer output.
        logits = self.final_layer(x)  # (batch_size, target_len, target_vocab_size)

        try:
            # Drop the keras mask, so it doesn't scale the losses/metrics.
            # b/250038731
            del logits._keras_mask
        except AttributeError:
            pass

        # Return the final output and the attention weights.
        return logits

# Training

## Training functions

In [ ]:
def masked_loss(label, pred):
    mask = label != 0
    loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
    loss = loss_object(label, pred)

    mask = tf.cast(mask, dtype=loss.dtype)
    loss *= mask

    loss = tf.reduce_sum(loss)/tf.reduce_sum(mask)
    return loss


def masked_accuracy(label, pred):
    pred = tf.argmax(pred, axis=2)
    label = tf.cast(label, pred.dtype)
    match = label == pred

    mask = label != 0

    match = match & mask

    match = tf.cast(match, dtype=tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    return tf.reduce_sum(match)/tf.reduce_sum(mask)

In [ ]:
# Used for transformer
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super().__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    step = tf.cast(step, dtype=tf.float32)
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)
  
  def get_config(self):
      config = {
      'd_model': self.d_model,
      'warmup_steps': self.warmup_steps,

        }
      return config

## Models setup (with hyper-pars)

In [ ]:
# RNN
RNN_UNITS = 256
modelRNN = Translator(RNN_UNITS, abs_text_processor, tit_text_processor, GRU=False)


In [ ]:
# GRU
GRU_UNITS = 256
modelGRU = Translator(GRU_UNITS, abs_text_processor, tit_text_processor, GRU=True)

In [ ]:
# Transformer hyper pars
num_layers = 1
d_model = 128
dff = 256
num_heads = 4 # was 8
dropout_rate = 0.1

# Training pars
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

# Load model
modelT = Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=len(abs_text_processor.get_vocabulary()),
    target_vocab_size=len(tit_text_processor.get_vocabulary()),
    dropout_rate=dropout_rate)

## Compile

In [ ]:
modelGRU.compile(
    loss=masked_loss,
    optimizer='Adam',
    metrics=[masked_accuracy])

In [ ]:
modelRNN.compile(
    loss=masked_loss,
    optimizer='Adam',
    metrics=[masked_accuracy])

In [ ]:
modelT.compile(
    loss=masked_loss,
    optimizer=optimizer,
    metrics=[masked_accuracy])

## Train RNN

In [ ]:
with tf.device('/device:GPU:0'):
    historyRNN = modelRNN.fit(train_ds,
                            epochs=30,
                            validation_data=val_ds,
                            callbacks=tf.keras.callbacks.EarlyStopping(
                                monitor='val_loss',
                                patience=3))

## Train GRU

In [ ]:
with tf.device('/device:GPU:0'):
    historyGRU = modelGRU.fit(train_ds,
                            epochs=30,
                            validation_data=val_ds,
                            callbacks=tf.keras.callbacks.EarlyStopping(
                                monitor='val_loss',
                                patience=3))

## Train Transformer

In [ ]:
with tf.device('/device:GPU:0'):
    historyT = modelT.fit(train_ds,
                            epochs=30,
                            validation_data=val_ds,
                            callbacks=tf.keras.callbacks.EarlyStopping(
                                monitor='val_loss',
                                patience=3))

## Training summary

In [ ]:
def plot_history(history, model_name):
    # Acc's
    plt.plot(history.history['masked_accuracy'])
    plt.plot(history.history['val_masked_accuracy'])
    plt.title(f'model accuracy - {model_name}')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()
    
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(f'model loss - {model_name}')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

### RNN

In [ ]:
modelRNN.summary()

In [ ]:
plot_history(historyRNN, f'RNN ({RNN_UNITS})')

### GRU

In [ ]:
modelGRU.summary()

In [ ]:
plot_history(historyGRU, f'GRU ({GRU_UNITS})')

### Transformer

In [ ]:
modelT.summary()

In [ ]:
# for naming
pars = dict(L = num_layers, emb = d_model, ff = dff, heads = num_heads)

s = ''
for k, v in pars:
    s += f'{k}={v}; '
s = s[:-2]

# plot history
plot_history(historyGRU, f'Transformer ({s})')

# Inference

1. Add inference from both branches on git, as implemented. Adjust variable names with above.
2. Use same sample from test-set



In [ ]:
def print_translation(abs, pred, ground_truth):
    print(f'{"Input:":15s}: {abs}')
    print(f'{"Prediction":15s}: {pred}')
    print(f'{"Ground truth":15s}: {ground_truth}')

In [ ]:
samples = random.sample(range(1087), 6)
print(samples)
rnn_x1, rnn_y1 = test_text[samples[0]], test_labels[samples[0]]
rnn_x2, rnn_y2 = test_text[samples[1]], test_labels[samples[1]]
gru_x1, gru_y1 = test_text[samples[2]], test_labels[samples[2]]
gru_x2, gru_y2 = test_text[samples[3]], test_labels[samples[3]]
tr_x1, tr_y1 = test_text[samples[4]], test_labels[samples[4]]
tr_x2, tr_y2 = test_text[samples[5]], test_labels[samples[5]]

In [ ]:
x1 = "we propose a solution to the annotation bottleneck for statistical parsing , by exploiting the lexicalized nature of combinatory categorial grammar ( ccg ) . the parsing model uses predicate-argument dependencies for training , which are derived from sequences of ccg lexical categories rather than full derivations . a simple method is used for extracting dependencies from lexical category sequences , resulting in high precision , yet incomplete and noisy data . the dependency parsing model of clark and curran ( 2004b ) is extended to exploit this partial training data . remarkably , the accuracy of the parser trained on data derived from category sequences alone is only 1.3 % worse in terms of f-score than the parser trained on complete dependency structures ."
x2 = "this paper describes a probabilistic model for coordination disambiguation integrated into syntactic and case structure analysis . our model probabilistically assesses the parallelism of a candidate coordinate structure using syntactic/semantic similarities and cooccurrence statistics . we integrate these probabilities into the framework of fully-lexicalized parsing based on largescale case frames . this approach simultaneously addresses two tasks of coordination disambiguation : the detection of coordinate conjunctions and the scope disambiguation of coordinate structures . experimental results on web sentences indicate the effectiveness of our approach ."
y1 = "partial training for a lexicalized-grammar parser"
y2 = "probabilistic coordination disambiguation in a fully-lexicalized japanese parser"

## RNN

In [ ]:
rnn_yhat1 = modelRNN.translate([x1])
rnn_yhat2 = modelRNN.translate([x2])

print_translation(x1, rnn_yhat1[0].numpy().decode(), y1)
print_translation(x2, rnn_yhat2[0].numpy().decode(), y2)

## GRU

In [ ]:
gru_yhat1 = modelGRU.translate([x1])
gru_yhat2 = modelGRU.translate([x2])

print_translation(x1, gru_yhat1[0].numpy().decode(), y1)
print_translation(x2, gru_yhat2[0].numpy().decode(), y2)

## Transformer

In [ ]:
class TranslatorT(tf.Module):
  def __init__(self, tokenizer, detokenizer, transformer):
    self.tokenizer = tokenizer
    self.detokenizer = detokenizer
    self.transformer = transformer

  def __call__(self, sentence):
    # The input sentence is an abstract, hence adding the `[START]` and `[END]` tokens.
    assert isinstance(sentence, tf.Tensor)
    if len(sentence.shape) == 0:
      sentence = sentence[tf.newaxis]

    # TODO: use our own tokenizer
    sentence = self.tokenizer(sentence).to_tensor()

    encoder_input = sentence

    # As the output language a title, initialize the output with the
    # `[START]` token.
    # TODO: look up how to tokenize this
    start_end = self.detokenizer([''])[0]
    start = start_end[0][tf.newaxis]
    end = start_end[1][tf.newaxis]

    # `tf.TensorArray` is required here (instead of a Python list), so that the
    # dynamic-loop can be traced by `tf.function`.
    output_array = tf.TensorArray(dtype=tf.int64, size=0, dynamic_size=True)
    output_array = output_array.write(0, start)

    # 50 is an arbitrary break-point
    for i in tf.range(50):
      output = tf.transpose(output_array.stack())
      predictions = self.transformer([encoder_input, output], training=False)

      # Select the last token from the `seq_len` dimension.
      predictions = predictions[:, -1:, :]  # Shape `(batch_size, 1, vocab_size)`.

      predicted_id = tf.argmax(predictions, axis=-1)

      # Concatenate the `predicted_id` to the output which is given to the
      # decoder as its input.
      output_array = output_array.write(i+1, predicted_id[0])

      if predicted_id == end:
        break

    output = tf.transpose(output_array.stack())

    tit_vocab = np.array(self.detokenizer.get_vocabulary())
    tokens = tit_vocab[output.numpy()]
    text = [' '.join(tok) for tok in tokens]

    # The output shape is `(1, tokens)`.
    # text = self.detokenizer(output)[0]  # Shape: `()`.

    #tokens = self.detokenizer lookup(output)[0]

    # `tf.function` prevents us from using the attention_weights that were
    # calculated on the last iteration of the loop.
    # So, recalculate them outside the loop.
    self.transformer([encoder_input, output[:,:-1]], training=False)
    attention_weights = self.transformer.decoder.last_attn_scores

    # return text, tokens, attention_weights
    return text, attention_weights


In [ ]:
transT = TranslatorT(abs_text_processor, tit_text_processor, modelT)

In [ ]:
t_yhat1, _ = transT(tf.constant(x1))
t_yhat2, _ = transT(tf.constant(x2))

t_yhat1 = t_yhat1[0].replace('[START] ', '').replace(' [END]', '')
t_yhat2 = t_yhat2[0].replace('[START] ', '').replace(' [END]', '')

print_translation(x1, t_yhat1, y1)
print_translation(x2, t_yhat2, y2)

# BertScore

See code snippit Samuel on teams

In [ ]:
!pip install git+https://github.com/Tiiiger/bert_score
!pip install transformers

In [ ]:
from bert_score import BERTScorer
import logging
import transformers

In [ ]:
scorer = BERTScorer(lang="en", rescale_with_baseline=True)

In [ ]:
# Sanity checking:
generated_titles = [
    'example generated title 1',
    'example generated title 2',
    'example generated title 3'
    ]

actual_titles = [
    'example actual title 1',
    'example actual title 2',
    'example actual title 3'
    ]

# Precision, Recall, F1 scores 
# len(actual_titles) == len(generated_titles) == len(P) == len(R) == len(F1)
P, R, F1 = scorer.score(generated_titles, actual_titles)

In [ ]:
rnn_preds, gru_preds, tra_preds = [], [], []
counter = 0
for text in test_text:
    rnn_preds.append(modelRNN.translate([text])[0].numpy().decode())
    gru_preds.append(modelGRU.translate([text])[0].numpy().decode())
    
    tp, _ = transT(tf.constant(text))
    tp = tp[0].replace('[START] ', '').replace(' [END]', '')
    
    tra_preds.append(tp)

    counter+=1
    if counter % 100 == 0:
        print(f'{counter/len(test_labels)}%')

In [ ]:
rnn_P, rnn_R, rnn_F1 = scorer.score(rnn_preds, test_labels)

In [ ]:
gru_P, gru_R, gru_F1 = scorer.score(gru_preds, test_labels)

In [ ]:
tra_P, tra_R, tra_F1 = scorer.score(tra_preds, test_labels)

In [ ]:
print("RNN\t\tP mean: {},\tR mean: {},\tF1 mean: {}".format(rnn_P.mean(), rnn_R.mean(), rnn_F1.mean()))
print("GRU\t\tP mean: {},\tR mean: {},\tF1 mean: {}".format(gru_P.mean(), gru_R.mean(), gru_F1.mean()))
print("Transformer\tP mean: {},\tR mean: {},\tF1 mean: {}".format(tra_P.mean(), tra_R.mean(), tra_F1.mean()))

In [ ]:
print("RNN\t\tP std: {},\tR std: {},\tF1 std: {}".format(rnn_P.std(), rnn_R.std(), rnn_F1.std()))
print("GRU\t\tP std: {},\tR std: {},\tF1 std: {}".format(gru_P.std(), gru_R.std(), gru_F1.std()))
print("Transformer\tP std: {},\tR std: {},\tF1 std: {}".format(tra_P.std(), tra_R.std(), tra_F1.std()))

## Best performers

Below shows the top-10 best performing titles per model.

In [ ]:
def print_topk(header, vals, idxs, preds):
  print("{}".format(header))
  [print_top(idx, v, test_text, test_labels, preds) for v, idx in zip(vals, idxs)]

def print_top(idx, v, inps, lbl, prds):
  print("Idx {} ({})".format(idx, v))
  print("True: {}\nPred: {}".format(lbl[idx], prds[idx]))
  print("Abst: {}\n".format(inps[idx]))

### RNN

In [ ]:
vals, idxs = rnn_F1.topk(10)
print_topk("RNN", vals, idxs, tra_preds)

### GRU

In [ ]:
vals, idxs = gru_F1.topk(10)
print_topk("GRU", vals, idxs, tra_preds)

### Transformer

In [ ]:
vals, idxs = tra_F1.topk(10)
print_topk("Transformer", vals, idxs, tra_preds)

## Histograms

In [ ]:
plt.hist(rnn_F1)
plt.plot()

In [ ]:
plt.hist(gru_F1)
plt.plot()

In [ ]:
plt.hist(tra_F1)
plt.plot()

# Save data

In [ ]:
import torch

In [ ]:
torch.save(gru_P, "rnn_P.pt")
torch.save(gru_P, "gru_P.pt")
torch.save(gru_P, "tra_P.pt")

In [ ]:
torch.save(gru_R, "rnn_R.pt")
torch.save(gru_R, "gru_R.pt")
torch.save(gru_R, "tra_R.pt")

In [ ]:
torch.save(gru_F1, "rnn_F1.pt")
torch.save(gru_F1, "gru_F1.pt")
torch.save(gru_F1, "tra_F1.pt")

In [ ]:
torch.save(rnn_preds, "rnn_preds.pt")
torch.save(gru_preds, "gru_preds.pt")
torch.save(tra_preds, "tra_preds.pt")

In [ ]:
modelRNN.save("rnn.model")

In [ ]:
modelGRU.save("gru.model")

In [ ]:
modelT.save("transformer.model")

## Zip for download

Cannot download dirs from colab, so zip contents first, then download the archive.

In [ ]:
import shutil

In [ ]:
shutil.make_archive('content', 'zip', '.')